In [25]:
library(glmnet)
library(dplyr)

In [26]:
set.seed(42) #selecionar inicio de valores aleatorios

In [32]:
data(iris)
df_iris <- iris 

In [47]:
head(df_iris)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,5.1,3.5,1.4,0.2,setosa
2,4.9,3.0,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa
4,4.6,3.1,1.5,0.2,setosa
5,5.0,3.6,1.4,0.2,setosa
6,5.4,3.9,1.7,0.4,setosa


In [34]:
df_iris$Species <- as.factor(df_iris$Species)

In [35]:
str(df_iris)

'data.frame':	150 obs. of  5 variables:
 $ Sepal.Length: num  5.1 4.9 4.7 4.6 5 5.4 4.6 5 4.4 4.9 ...
 $ Sepal.Width : num  3.5 3 3.2 3.1 3.6 3.9 3.4 3.4 2.9 3.1 ...
 $ Petal.Length: num  1.4 1.4 1.3 1.5 1.4 1.7 1.4 1.5 1.4 1.5 ...
 $ Petal.Width : num  0.2 0.2 0.2 0.2 0.2 0.4 0.3 0.2 0.2 0.1 ...
 $ Species     : Factor w/ 3 levels "setosa","versicolor",..: 1 1 1 1 1 1 1 1 1 1 ...


In [36]:
n <- nrow(df_iris)
n

[1] 150

In [43]:
partition <- sample(seq_len(n), size = 0.7 * n)
partition

[1]  43  42 149  97  25 115  32  81  14 111   6 150 113 138  31  94 140  38
 [19]  95 134  84  15  34 148  87  60  12  26  41  65  66  56  24  98 146  92
 [37] 107  61  62 142 128 120 132 144  27  10  57  28  37 105   5  35  78 141
 [55] 103  54  90 136 139  52 143  59 106  30  91 130  75 135  73  17 112  13
 [73]  63  49 137   1  83  33  96   2  93   8   3  79 124 121 126  51 125  70
 [91] 102  47 119 131  55  16 101   7   9 122 116  68  18  89  72

In [46]:
training <- df_iris[partition,]
head(training)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
43,4.4,3.2,1.3,0.2,setosa
42,4.5,2.3,1.3,0.3,setosa
149,6.2,3.4,5.4,2.3,virginica
97,5.7,2.9,4.2,1.3,versicolor
25,4.8,3.4,1.9,0.2,setosa
115,5.8,2.8,5.1,2.4,virginica


In [45]:
test <- df_iris[-partition,]
head(test)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
4,4.6,3.1,1.5,0.2,setosa
11,5.4,3.7,1.5,0.2,setosa
19,5.7,3.8,1.7,0.3,setosa
20,5.1,3.8,1.5,0.3,setosa
21,5.4,3.4,1.7,0.2,setosa
22,5.1,3.7,1.5,0.4,setosa


In [49]:
num_cols <- sapply(training, is.numeric)
num_cols

Sepal.Length  Sepal.Width Petal.Length  Petal.Width      Species 
        TRUE         TRUE         TRUE         TRUE        FALSE

In [50]:
training_mean <- sapply(training[, num_cols, drop = FALSE], mean)
training_mean

Sepal.Length  Sepal.Width Petal.Length  Petal.Width 
    5.852381     3.058095     3.785714     1.216190

In [51]:
training_sds <- sapply(training[, num_cols, drop = FALSE], sd)
training_sds

Sepal.Length  Sepal.Width Petal.Length  Petal.Width 
   0.8220630    0.4347371    1.7428662    0.7693808

In [57]:
normalization_ap <- function(data, mean, standard_deviation, number_c) {
    out <- data
    out[, number_c] <- sweep(out[, number_c, drop = FALSE], 2, mean, `-`)
    out[, number_c] <- sweep(out[, number_c, drop = FALSE], 2, standard_deviation, `/`)
    out
}

In [61]:
training_normalization <- normalization_ap(training, training_mean, training_sds, num_cols)
head(training_normalization)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
43,-1.7667514,0.3264151,-1.4262221,-1.3207899,setosa
42,-1.6451063,-1.7438016,-1.4262221,-1.1908153,setosa
149,0.4228618,0.7864633,0.9262247,1.4086775,virginica
97,-0.1853641,-0.3636571,0.2377037,0.1089311,versicolor
25,-1.2801707,0.7864633,-1.0819616,-1.3207899,setosa
115,-0.0637189,-0.5936812,0.7540945,1.5386522,virginica


In [63]:
test_normalization <- normalization_ap(test, training_mean, training_sds, num_cols)
head(test_normalization)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
4,-1.5234611,0.09639104,-1.311469,-1.320790,setosa
11,-0.5502996,1.47653554,-1.311469,-1.320790,setosa
19,-0.1853641,1.70655962,-1.196715,-1.190815,setosa
20,-0.9152352,1.70655962,-1.311469,-1.190815,setosa
21,-0.5502996,0.78646329,-1.196715,-1.320790,setosa
22,-0.9152352,1.47653554,-1.311469,-1.060841,setosa


In [64]:
A <- training

In [65]:
A[,1] <-(training[,1]-training_mean[1]/training_sds[1])
A[,2] <-(training[,1]-training_mean[1]/training_sds[1])
A[,3] <-(training[,1]-training_mean[1]/training_sds[1])
A[,4] <-(training[,1]-training_mean[1]/training_sds[1])

In [67]:
x_training <- as.matrix(training_normalization[, num_cols, drop = FALSE])
x_test <- as.matrix(test_normalization[, num_cols, drop = FALSE])
y_training <- training_normalization$Species
y_test <- test_normalization$Species

In [97]:
cv_fit <- cv.glmnet(x = x_training,
                  y = y_training,
                   family = "multinomial",
                   type.measure = "class",
                   alpha = 0.5,
                   nfolds = 5)
cv_fit


Call:  cv.glmnet(x = x_training, y = y_training, type.measure = "class",      nfolds = 5, family = "multinomial", alpha = 0.5) 

Measure: Misclassification Error 

      Lambda Measure      SE Nonzero
min 0.000201 0.01905 0.01166       4
1se 0.007556 0.02857 0.01905       3

In [98]:
best_lambda <- cv_fit$lambda.1se
best_lambda

[1] 0.007556201

In [84]:
probs_glmnet <- predict(cv_fit,
                       newx = x_test,
                       s = best_lambda,
                       type = "response")[,,1]

In [85]:
dim(x_test)

[1] 45  4

In [87]:
head(probs_glmnet)

,setosa,versicolor,virginica
4,0.9642528,0.03574500,2.148093e-06
11,0.9768408,0.02315819,9.587338e-07
19,0.9633442,0.03665316,2.640316e-06
20,0.9847226,0.01527637,9.872122e-07
21,0.9454431,0.05455375,3.195809e-06
22,0.9778192,0.02217864,2.184701e-06


In [90]:
max.col(probs_glmnet, ties.method = "first")

[1] 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 3 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3
[39] 3 3 3 3 3 3 3

In [93]:
pred_glmnet <- colnames(probs_glmnet)[max.col(probs_glmnet, ties.method = "first")]

In [95]:
pred_glmnet <- factor(pred_glmnet, levels = levels(y_test))

In [96]:
pred_glmnet

[1] setosa     setosa     setosa     setosa     setosa     setosa    
 [7] setosa     setosa     setosa     setosa     setosa     setosa    
[13] setosa     setosa     setosa     setosa     versicolor versicolor
[19] versicolor versicolor versicolor virginica  versicolor versicolor
[25] versicolor versicolor versicolor versicolor versicolor versicolor
[31] versicolor versicolor virginica  virginica  virginica  virginica 
[37] virginica  virginica  virginica  virginica  virginica  virginica 
[43] virginica  virginica  virginica 
Levels: setosa versicolor virginica